# Zooming-in the spectrum by decimating
This example uses the same connections that were used for demo 0. In this case, one of the channels of the Analysis Polyphase Filter Bank is used. Mixing with the second-stage fine down-converter allows to center the frequency of interest in that channel. Once the bin is at the center of the channel, additional decimation allows zooming-in the spectrum.

I used this notebook with some external components:
* Noise source: Agilent 346B (with some amplifiers to make it bigger).
* RF splitter/combiner: TB-EP4RKUC+.
* Input: ADC_D
* Output: DAC_B

### Receiving chain
The Analyzer is build with three main blocks:
* Analysis PFB
* DDS-based mixer
* CIC filter

Each of the output channels of the PFB feed one independent DDS-based mixer. The mixer is a totally digital process, which means the output tone of this DDS is used to shift the channel in frequency by multiplying the signals. The advantage of using this technique is that the PFB provide a coarse grain down-conversion, and the second stage mixing provides fine-tunning by using sub-mHz. This is possible as a result of the DDS running at a speed that is much lower (512 times in this case) than that at the input of the PFB.

After the signal is centered with the fine-mixer, it can be decimated by a bigger factor to remove noise and improve the SNR. This step is done using a CIC filter, one per channel. Even if the CIC strcture applies one CIC decimator per channel (1024 total in this case), the decimation factor is the same across channels to simplify system design.

### Don't forget the DDS is shared...
The DDS block is shared between the reception and transmission chains. This saves a lot of resources and also provides an easy way of ensuring up-conversion and down-conversion are using the same frequency and phase. 

In practice this means that if channel k is used at the output side to generate a tone, and the same channel k is used at the input side, the DDS is basically the same. For zooming-in the spectrum what is needed is precisely to fine-tune the channel center at the input side using the second stage fine-mixer. The problem is that if the frequency is changed on the reception side, then it will overwrite the output signal because, again, the DDS is shared. To solve this issue, this example uses the following trick. 

The ADC and DAC units of the RFSoC have an internal mixer whose frequency is set independently. To decouple output and input PFB channels, this example sets the mixer frequency on the DAC to 1000 MHz, and the mixer frequency of the ADC to -950 MHz (the minus sign is to ensure the same side-band is recovered and avoid mirroring). The result is that, even if the same frequency is specified for the output and input, different PFB channels will be used for Analysis and Synthesis.

### Zomming-in
The provided software drivers were developed to take care of most of the details. To zoom-in the spectrum, the corresponding Analysis PFB channel will be recovered and the second-stage mixing process by-passed at a first approximation. The decimation factor of the CIC is set to 1 to take a broad look at the spectrum. The second stage mixer is by-passed by using source("input") when configuring the chain.

Once the signal is identified or a specific frequency location wants to be analyzed further, the option source("product") is set to apply the second-stage fine-mixing. Still a decimation of 1 can be used at a first approximation. The decimation factor of the CIC can then be increased to perform the effective zoom-in process, as a result of lowering the bandwidth of the resulting signal. The maximum CIC decimation is 250.

In [ ]:
import sys
sys.path.append('./soft')

from pfbs import *

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift

In [ ]:
# Initialize Firmware.
soc = TopSoc('./pfbs_v1.bit')

# Print information.
print(soc)

In [ ]:
################################################
### Initialize Analysis and Synthesis Chains ###
################################################

# Dual Chain: it includes both analysis and synthesis chains.
dual = KidsChain(soc, dual=soc['dual'][0])

# Analysis Chain: used as input for spectrum analyzer functionality.
analysis_ch = dual.analysis

# Synthesis Chain: used as output for signal generation.
synthesis_ch = dual.synthesis

In [ ]:
##################################
### Generate some output tones ###
##################################
# Disable all outputs.
synthesis_ch.alloff()

# Set DAC mixer frequency.
synthesis_ch.set_mixer_frequency(1000)

# Set quantization.
synthesis_ch.qout(2)

# Set tones.
f_v = np.linspace(1100,1110,5)
for f in f_v:
    print("f = {} MHz".format(f))
    synthesis_ch.set_tone_simple(f=f, g=0.5)

In [ ]:
#########################
### Spectrum Analyzer ###
#########################
# Set ADC mixer frequency.
analysis_ch.set_mixer_frequency(-950)

# Set data source.
analysis_ch.source("input") # by-pass dds product.

# Decimation.
analysis_ch.set_decimation(1)
analysis_ch.qout(2)

# Frequency range.
fstart = 1095
fend   = 1115
f = np.arange(fstart, fend, analysis_ch.fc_ch)
print("Spectrum")
print("fstart = {} MHz, fend = {} MHz, fc = {} MHz".format(fstart, fend, analysis_ch.fc_ch))

# Frequency and amplitude vectors.
FF = []
AA = []
plt.figure(dpi=150);
for i,fck in enumerate(f):
    print("i = {}, fck = {} MHz".format(i,fck))
    
    # Transfer data.
    [xi,xq] = analysis_ch.get_bin(fck,force_dds=True)
    x = xi + 1j*xq
    
    # Frequency vector.
    F = (np.arange(len(x))/len(x)-0.5)*analysis_ch.fs_ch    
    
    # Normalization factor.
    NF = (2**15)*len(F)

    w = np.hanning(len(x))
    xw = x*w
    YY = fftshift(fft(xw))
    YYlog = 20*np.log10(abs(YY)/NF)
    AA = np.concatenate((AA,YYlog))
    
    Fk = F+fck
    FF = np.concatenate((FF,Fk))
    plt.plot(Fk,YYlog);
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

In [ ]:
#########################
### Analyze 1 channel ###
#########################
# Frequency of interest
fck = 1110

# Get corresponding channel center frequency.
fckq = analysis_ch.ch2freq(analysis_ch.freq2ch(fck))

# Set data source.
analysis_ch.source("input")

# Decimation.
analysis_ch.set_decimation(1)

# Transfer data.
[xi,xq] = analysis_ch.get_bin(fckq,force_dds=True)
x1 = xi + 1j*xq
    
# Spectrum.
F = (np.arange(len(x1))/len(x1)-0.5)*analysis_ch.fs_ch    
w = np.hanning(len(x1))

# Normalization factor.
NF = (2**15)*len(F)

xw1 = x1*w
Y1 = fftshift(fft(xw1))
Y1log = 20*np.log10(abs(Y1)/NF)

plt.figure(dpi=150);
plt.plot(F+fckq,Y1log);
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
plt.savefig('spectrum_zoom_0.jpg')

In [ ]:
#######################
### Zoom-in channel ###
#######################
# Frequency of interest
fck = 1110

# Set data source.
analysis_ch.source("product")

# Decimation.
analysis_ch.set_decimation(1)

# Transfer data.
[xi,xq] = analysis_ch.get_bin(fck,force_dds=True)
x1 = xi + 1j*xq
    
# Spectrum.
F = (np.arange(len(x1))/len(x1)-0.5)*analysis_ch.fs_ch    
w = np.hanning(len(x1))

# Normalization factor.
NF = (2**15)*len(F)

xw1 = x1*w
Y1 = fftshift(fft(xw1))
Y1log = 20*np.log10(abs(Y1)/NF)

plt.figure(dpi=150);
plt.plot(F*1000,Y1log,label='D={}'.format(analysis_ch.decimation));
plt.legend()
plt.xlabel("Frequency [kHz]");
plt.ylabel("Amplitude [dB]");
plt.savefig('spectrum_zoom_dec_{}.jpg'.format(analysis_ch.decimation))

In [ ]:
#######################
### Zoom-in channel ###
#######################
# Frequency of interest
fck = 1110

# Set data source.
analysis_ch.source("product")

# Decimation.
analysis_ch.set_decimation(10)

# Transfer data.
[xi,xq] = analysis_ch.get_bin(fck,force_dds=True)
x1 = xi + 1j*xq
    
# Spectrum.
F = (np.arange(len(x1))/len(x1)-0.5)*analysis_ch.fs_ch    
w = np.hanning(len(x1))

# Normalization factor.
NF = (2**15)*len(F)

xw1 = x1*w
Y1 = fftshift(fft(xw1))
Y1log = 20*np.log10(abs(Y1)/NF)

plt.figure(dpi=150);
plt.plot(F*1000,Y1log,label='D={}'.format(analysis_ch.decimation));
plt.legend()
plt.xlabel("Frequency [kHz]");
plt.ylabel("Amplitude [dB]");
plt.savefig('spectrum_zoom_dec_{}.jpg'.format(analysis_ch.decimation))

In [ ]:
#######################
### Zoom-in channel ###
#######################
# Frequency of interest
fck = 1110

# Set data source.
analysis_ch.source("product")

# Decimation.
analysis_ch.set_decimation(250)

# Transfer data.
[xi,xq] = analysis_ch.get_bin(fck,force_dds=True)
x1 = xi + 1j*xq
    
# Spectrum.
F = (np.arange(len(x1))/len(x1)-0.5)*analysis_ch.fs_ch    
w = np.hanning(len(x1))

# Normalization factor.
NF = (2**15)*len(F)

xw1 = x1*w
Y1 = fftshift(fft(xw1))
Y1log = 20*np.log10(abs(Y1)/NF)

plt.figure(dpi=150);
plt.plot(F*1000,Y1log,label='D={}'.format(analysis_ch.decimation));
plt.legend()
plt.xlabel("Frequency [kHz]");
plt.ylabel("Amplitude [dB]");
plt.savefig('spectrum_zoom_dec_{}.jpg'.format(analysis_ch.decimation))